In [ ]:
# conversational_agent.py

import os
from typing import Dict, List, Any
from operator import itemgetter

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

# 工具导入
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.calculator.tool import CalculatorTool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory


def create_conversational_agent():
    """
    创建一个具备对话记忆功能的 Agent，使用本地 Ollama 的 qwen 模型
    """
    print("正在初始化 Ollama 模型...")
    # 1. 初始化 LLM
    llm = ChatOllama(
        model="qwen:4b",  # 使用本地 Ollama 的 qwen 模型
        temperature=0.1,  # 降低随机性
        timeout=120,  # 增加超时时间，防止复杂查询中断
    )
    print("模型初始化完成")

    # 2. 定义工具
    print("正在设置工具...")
    tools = [
        DuckDuckGoSearchRun(name="search",
                           description="用于在网络上搜索信息。当你需要查找最新信息、事实、新闻或网络上的内容时使用此工具。"),
        CalculatorTool(name="calculator",
                      description="用于执行数学计算。当你需要计算数学表达式时使用此工具。")
    ]
    print(f"工具设置完成，共 {len(tools)} 个工具可用")

    # 3. 创建 Agent 的提示模板
    print("正在创建 Agent 提示模板...")
    prompt = ChatPromptTemplate.from_messages([
        ("system", """你是一个智能助手，可以通过使用工具来帮助用户回答问题。

你可以使用以下工具：
1. search: 用于在网络上搜索信息
2. calculator: 用于执行数学计算

请遵循以下规则：
- 仔细分析用户的问题，判断是否需要使用工具
- 如果问题可以直接回答，就直接回答
- 如果需要查询信息或计算，使用适当的工具
- 回答要简洁、准确、有帮助
- 使用中文回复中文问题，使用英文回复英文问题
        """),
        MessagesPlaceholder(variable_name="chat_history"),  # 添加对话历史
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),  # Agent 的思考过程
    ])
    print("提示模板创建完成")

    # 4. 创建 Agent
    print("正在创建 Agent...")
    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    print("Agent 创建完成")

    # 5. 添加对话历史功能
    print("正在添加对话历史功能...")
    message_history = {}

    def get_session_history(session_id):
        if session_id not in message_history:
            message_history[session_id] = ChatMessageHistory()
        return message_history[session_id]

    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )
    print("对话历史功能添加完成")

    return agent_with_chat_history


def run_conversation(agent, session_id="default_user"):
    """
    运行一个交互式对话会话
    """
    print("\n=== 开始与 Agent 对话 (输入 'exit' 退出) ===")

    while True:
        user_input = input("\n你: ")
        if user_input.lower() == 'exit':
            break

        # 调用 Agent 并获取响应
        response = agent.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )

        print(f"\nAgent: {response['output']}")


def run_demo_conversation(agent, session_id="demo_user"):
    """
    运行预设的演示对话
    """
    demo_questions = [
        "今天的日期是什么？",
        "计算一下 (15 * 37) + 89 的结果",
        "LangChain 是什么？",
        "它的主要功能是什么？",  # 测试记忆功能，"它"指代 LangChain
        "计算 567 * 832 - 443 的结果"
    ]

    print("\n=== 演示对话开始 ===")

    for i, question in enumerate(demo_questions):
        print(f"\n问题 {i+1}: {question}")

        response = agent.invoke(
            {"input": question},
            config={"configurable": {"session_id": session_id}}
        )

        print(f"Agent 回答: {response['output']}")

    print("\n=== 演示对话结束 ===")


if __name__ == "__main__":
    # 创建 Agent
    agent = create_conversational_agent()

    # 选择运行模式
    print("\n请选择运行模式:")
    print("1. 演示模式 (运行预设问题)")
    print("2. 交互模式 (自由对话)")

    choice = input("请输入选择 (1/2): ")

    if choice == "1":
        run_demo_conversation(agent)
    else:
        run_conversation(agent)